### Demonstrate Q learning algorithm with suitable assumption for a problem statement
### Assumed problem statement : a 2D grid world where an agent needs to find the shortest path to a goal while avoiding obstacles.

In [1]:
import numpy as np

# Define the environment and Q-table
# 0: empty cell, 1: obstacle, 2: goal
env = np.array([[0, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 2]])
Q = np.zeros((np.prod(env.shape), 4))

# Define hyperparameters
lr = 0.8
dr = 0.95
ep = 0.2
ne = 1000

# Define A
A = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Perform Q-learning
for _ in range(ne):
    S = (0, 0)
    while env[S] != 2:
        action = np.random.choice(4) if np.random.uniform(0, 1) < ep else np.argmax(Q[S[0]*env.shape[1]+S[1]])
        new_S = (max(min(S[0]+A[action][0], env.shape[0]-1), 0), max(min(S[1]+A[action][1], env.shape[1]-1), 0))
        reward = -1 if env[new_S] == 1 else (10 if env[new_S] == 2 else 0)
        Q[S[0]*env.shape[1]+S[1]][action] += lr * (reward + dr*np.max(Q[new_S[0]*env.shape[1]+new_S[1]]) - 
                                 Q[S[0]*env.shape[1]+S[1]][action])
        S = new_S

# Find the optimal path
S = (0, 0)
optimal_path = [S]
while env[S] != 2:
    action = np.argmax(Q[S[0]*env.shape[1]+S[1]])
    S = (max(min(S[0]+A[action][0], env.shape[0]-1), 0), max(min(S[1]+A[action][1], env.shape[1]-1), 0))
    optimal_path.append(S)

print("Optimal Path:", optimal_path)

Optimal Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4)]


In [2]:
import numpy as np

# Define the environment (2D grid world)
# 0: empty cell, 1: obstacle, 2: goal
env = np.array([
    [0, 0, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 2]
])

# Define Q-table (state-action values)
num_states = np.prod(env.shape)
num_actions = 4  # Up, Down, Left, Right
Q = np.zeros((num_states, num_actions))

# Define hyperparameters
learning_rate = 0.8
discount_factor = 0.95
exploration_prob = 0.2
num_episodes = 1000

# Convert 2D grid to 1D state representation
def state_to_index(state):
    return state[0] * env.shape[1] + state[1]

# Perform Q-learning
for episode in range(num_episodes):
    state = (0, 0)  # Start from the top-left corner
    done = False

    while not done:
        if np.random.uniform(0, 1) < exploration_prob:
            action = np.random.choice(num_actions)  # Exploration
        else:
            action = np.argmax(Q[state_to_index(state)])  # Exploitation

        new_state = state
        if action == 0:  # Up
            new_state = (max(state[0] - 1, 0), state[1])
        elif action == 1:  # Down
            new_state = (min(state[0] + 1, env.shape[0] - 1), state[1])
        elif action == 2:  # Left
            new_state = (state[0], max(state[1] - 1, 0))
        elif action == 3:  # Right
            new_state = (state[0], min(state[1] + 1, env.shape[1] - 1))

        if env[new_state] == 1:
            reward = -1  # Penalty for hitting an obstacle
        elif env[new_state] == 2:
            reward = 10  # Reward for reaching the goal
            done = True
        else:
            reward = 0  # No immediate reward for other states

        Q[state_to_index(state)][action] = Q[state_to_index(state)][action] + learning_rate * (
            reward + discount_factor * np.max(Q[state_to_index(new_state)]) - Q[state_to_index(state)][action])

        state = new_state

# Find the optimal path
state = (0, 0)
optimal_path = [state]
while state != (3, 4):  # Goal state
    action = np.argmax(Q[state_to_index(state)])
    if action == 0:
        state = (max(state[0] - 1, 0), state[1])
    elif action == 1:
        state = (min(state[0] + 1, env.shape[0] - 1), state[1])
    elif action == 2:
        state = (state[0], max(state[1] - 1, 0))
    elif action == 3:
        state = (state[0], min(state[1] + 1, env.shape[1] - 1))
    optimal_path.append(state)

print("Optimal Path:")
for row in env:
    print(row)

print("Q-table:")
print(Q)

print("Optimal Path:")
for state in optimal_path:
    print(state)

Optimal Path:
[0 0 0 1 0]
[0 1 0 1 0]
[0 1 0 1 0]
[0 0 0 1 2]
Q-table:
[[ 6.24828107  5.93586702  6.24828107  6.57713797]
 [ 6.57713797  5.92330312  6.24828107  6.92330312]
 [ 6.92330312  7.2876875   6.57713797  7.1450625 ]
 [ 7.13148793  7.56566689  6.87816032  8.57375   ]
 [ 8.57371218  9.025       7.1450625   8.57361483]
 [ 6.24828107  5.62599421  5.88967384  5.89941801]
 [ 6.56899291  6.13529146  5.89717466  7.2876875 ]
 [ 6.92330312  7.67125     5.92330312  7.57375   ]
 [ 7.14506167  8.025       7.2424865   9.025     ]
 [ 8.57375     9.5         7.57374991  9.025     ]
 [ 5.93580696  0.          0.          4.22894737]
 [ 5.83821894  7.67125     4.44743819  6.099     ]
 [ 7.2876875   8.075       6.28768744  8.025     ]
 [ 7.57375     8.5         7.671174    9.5       ]
 [ 9.025      10.          8.025       9.5       ]
 [ 5.61390044  0.          0.          0.        ]
 [ 6.25718834  7.60988     5.063134    8.075     ]
 [ 7.67125     8.075       7.67125     8.5       ]
 [ 8.025   